In [1]:
#import potřebných knihoven
import numpy as np
import matplotlib.pyplot as plt
import pysheds
import geopandas as gpd
from osgeo import gdal

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
#nastavení workspace a základních proměnných
dem_file = 'path/to/your/dem.tif'
dem_ds = gdal.Open(dem_file)
dem_array = dem_ds.GetRasterBand(1).ReadAsArray()

In [ ]:
#vytvoření instance řeky
dem = pysheds.grid.from_raster(dem_array, data_name='dem')

In [ ]:
#vytvoření toku řeky
dem.resolve_flats('raise')
dem.fill_depressions('depression')
dem.flowdir()

In [ ]:
# Výpočet akumulace toku
acc = dem.accumulation()


In [5]:
# Vizualizace výsledků
plt.figure(figsize=(10, 10))
plt.imshow(acc, cmap='Blues', interpolation='none')
plt.colorbar(label='Flow Accumulation')
plt.title('Flow Accumulation Map')
plt.show()


Zvýšení hladiny moří: 0.4 metrů
